In [2]:
import os
import glob
import shutil

import SimpleITK as sitk
import ants

print(f"AntsPy version: {ants.__version__}")
print(f"SimpleITK version: {sitk.__version__}")

AntsPy version: 0.4.2
SimpleITK version: 2.4.1


In [3]:
import matplotlib.pyplot as plt

from ipywidgets import interact
import numpy as np
import SimpleITK as sitk
import cv2


def explore_3D_array(arr: np.ndarray, cmap: str = "gray"):
    """
    Given a 3D array with shape (Z,X,Y) This function will create an interactive
    widget to check out all the 2D arrays with shape (X,Y) inside the 3D array.
    The purpose of this function to visual inspect the 2D arrays in the image.

    Args:
      arr : 3D array with shape (Z,X,Y) that represents the volume of a MRI image
      cmap : Which color map use to plot the slices in matplotlib.pyplot
    """

    def fn(SLICE):
        plt.figure(figsize=(7, 7))
        plt.imshow(arr[SLICE, :, :], cmap=cmap)
        plt.show()

    interact(fn, SLICE=(0, arr.shape[0] - 1))

In [4]:
BASE_DIR = os.path.dirname(os.path.abspath("__file__"))
raw_img_path = os.path.join(
    BASE_DIR,
    "data",
    "ADNI1-T1-AD-CN",
    "Image",
    "Preprocess",
    "4_step_relevant_slices",
    "I10027.nii.gz",
)
template = os.path.join(BASE_DIR, "data", "Template", "MNI152_T1_1mm_Brain.nii")

# Basics

## AntsPy

In [5]:
import nibabel as nib

image = nib.load(raw_img_path)
image_data = image.get_fdata()
image_data.shape

(32, 182, 182)

In [11]:
raw_img_ants = ants.image_read(raw_img_path)

In [12]:
image_data = raw_img_ants.numpy()
image_data.shape

(32, 182, 182)

In [13]:
print(raw_img_ants)

ANTsImage (RAI)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (32, 182, 182)
	 Spacing    : (1.0, 1.0, 1.0)
	 Origin     : (0.0, 0.0, 0.0)
	 Direction  : [1. 0. 0. 0. 1. 0. 0. 0. 1.]



In [14]:
raw_img_ants_arr = raw_img_ants.numpy()

print(f"Type: {type(raw_img_ants_arr)}")
print(f"Shape: {raw_img_ants_arr.shape}")

Type: <class 'numpy.ndarray'>
Shape: (32, 182, 182)


In [15]:
explore_3D_array(arr=raw_img_ants_arr)

interactive(children=(IntSlider(value=15, description='SLICE', max=31), Output()), _dom_classes=('widget-inter…

In [23]:
explore_3D_array(arr=raw_img_ants_arr)

interactive(children=(IntSlider(value=95, description='SLICE', max=191), Output()), _dom_classes=('widget-inte…

In [60]:
raw_img_ants = ants.image_read(raw_img_path, reorient="ASR")
raw_img_ants_arr = raw_img_ants.numpy()

print(f"Type: {type(raw_img_ants_arr)}")
print(f"Shape: {raw_img_ants_arr.shape}")
explore_3D_array(arr=raw_img_ants_arr)

Type: <class 'numpy.ndarray'>
Shape: (218, 182, 182)


interactive(children=(IntSlider(value=108, description='SLICE', max=217), Output()), _dom_classes=('widget-int…

In [ ]:
raw_img_ants = ants.image_read(raw_old, reorient="ASR")
raw_img_ants_arr = raw_img_ants.numpy()

print(f"Type: {type(raw_img_ants_arr)}")
print(f"Shape: {raw_img_ants_arr.shape}")
explore_3D_array(arr=raw_img_ants_arr)

Type: <class 'numpy.ndarray'>
Shape: (193, 193, 32)


interactive(children=(IntSlider(value=15, description='SLICE', max=31), Output()), _dom_classes=('widget-inter…

In [ ]:
print(
    raw_img_ants[
        35:115,
        :,
    ].shape
)

In [ ]:
raw_img_ants_ = ants.from_numpy(
    raw_img_ants[
        50:114,
        :,
    ]
)
raw_img_ants_

In [179]:
filin = ants.resample_image(raw_img_ants_, (2, 2, 2), False, 1)

In [ ]:
print(filin)

## SimpleITK

In [163]:
def show_sitk_img_info(img: sitk.Image):
    """
    Given a sitk.Image instance prints the information about the MRI image contained.

    Args:
      img : instance of the sitk.Image to check out
    """
    pixel_type = img.GetPixelIDTypeAsString()
    origin = img.GetOrigin()
    dimensions = img.GetSize()
    spacing = img.GetSpacing()
    direction = img.GetDirection()

    info = {
        "Pixel Type": pixel_type,
        "Dimensions": dimensions,
        "Spacing": spacing,
        "Origin": origin,
        "Direction": direction,
    }
    for k, v in info.items():
        print(f" {k} : {v}")

In [161]:
raw_img_sitk = sitk.ReadImage(raw_img_path, sitk.sitkFloat32)

In [ ]:
show_sitk_img_info(raw_img_sitk)

In [ ]:
raw_img_sitk_arr = sitk.GetArrayFromImage(raw_img_sitk)

print(f"Type: {type(raw_img_sitk_arr)}")
print(f"Shape: {raw_img_sitk_arr.shape}")

# Image Orientation

## AntsPy is Z, Y, X and the same in numpy

In [ ]:
raw_img_ants = ants.image_read(raw_img_path, reorient="LSP")
print(raw_img_ants)

In [ ]:
print(raw_img_ants.get_orientation())

In [ ]:
ants.plot(raw_img_ants, figsize=2, axis=2)

## Simple ITK  is X, Y, Z and in the numpy is Z, Y, X

In [11]:
raw_img_sitk = sitk.ReadImage(raw_img_path, sitk.sitkFloat32)
raw_img_sitk = sitk.DICOMOrient(raw_img_sitk, "RPS")

In [ ]:
raw_img_sitk_arr = sitk.GetArrayFromImage(raw_img_sitk)

print(f"Type: {type(raw_img_sitk_arr)}")
print(f"Shape: {raw_img_sitk_arr.shape}")

# Registration 

In [13]:
raw_img_ants = ants.image_read(raw_img_path, reorient="IAL")
template = ants.image_read(template, reorient="IAL")

In [ ]:
ants.plot(raw_img_ants, figsize=2, axis=2)

In [ ]:
print(raw_img_ants)
print(template)

In [ ]:
transformation = ants.registration(
    fixed=template, moving=raw_img_ants, type_of_transform="SyN", verbose=True
)

In [ ]:
transformation

In [187]:
registered_img_ants = transformation["warpedmovout"]

In [ ]:
filin = ants.resample_image(registered_img_ants, (2, 2, 2), False, 1)
print(filin)

In [ ]:
ants.plot(filin, figsize=2, axis=2)

In [ ]:
ants.plot(registered_img_ants, figsize=2, axis=2)

In [ ]:
ants.plot(registered_img_ants, figsize=2, axis=2)

In [ ]:
transformation = ants.registration(
    fixed=template, moving=raw_img_ants, type_of_transform="SyN", verbose=True
)

In [ ]:
# registered_img_ants.to_file(.nii.gz)

In [ ]:
registered_mask_img_ants = ants.apply_transforms(
    moving=mask_img_ants,  # <-- update
    fixed=transformation["warpedmovout"],
    transformlist=transformation["fwdtransforms"],
    verbose=True,
)

# Skull-stripping

In [197]:
raw_img_path = os.path.join(
    BASE_DIR, "data", "ADNI1", "ADNI1-Screening-Nifti", "002_S_0295.nii.gz"
)

In [198]:
brain_path = os.path.join(BASE_DIR, "data", "ADNI1", "Teste", "002_S_0295_brain.nii.gz")
mask_paths = os.path.join(BASE_DIR, "data", "ADNI1", "Teste", "002_S_0295_mask.nii.gz")
tiv_paths = os.path.join(BASE_DIR, "data", "ADNI1", "Teste", "002_S_0295.csv")

In [199]:
from deepbet import run_bet

input_paths = [raw_img_path]
brain_paths = [brain_path]
mask_paths = [mask_paths]
tiv_paths = [tiv_paths]
run_bet(
    input_paths,
    brain_paths,
    mask_paths,
    tiv_paths,
    threshold=0.5,
    n_dilate=0,
    no_gpu=False,
)

In [ ]:
raw_img_ants = ants.image_read(brain_paths[0])
ants.plot(raw_img_ants, figsize=2, axis=2)